In [1]:
%pip install pandas dbfread


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: C:\Users\anves\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [4]:
from dbfread import DBF
import pandas as pd

# Path to your .dbf file
dbf_file_path = 'C:\Anna_Data_D_files\sem6\z_capstone\Agriculture\A_Phase_1\datasets\DiCRA\Telengana\SOCIO-ECONOMIC\CROP_TYPE\croptype_telangana_2014_15.img.tif.vat.dbf'

# Read the .dbf file
dbf = DBF(dbf_file_path)

# Convert to DataFrame
df = pd.DataFrame(iter(dbf))

# Display the DataFrame
print(df)

csv_file_path = 'C:\Anna_Data_D_files\sem6\z_capstone\Agriculture\A_Phase_2\code- review2\preprocessed_files\DICRA\Crop_Type_2015_dbf.csv'
df.to_csv(csv_file_path, index=False)


    Value      Count                        Class_Name  Red  Green  Blue  \
0       0  1604594.0                                      0      0     0   
1       1    77223.0         1. Irrigated-DC-rice-rice    0    255     0   
2       2   209043.0       2. Irrigated-DC-rice-pulses  160     82    45   
3       3    83203.0  3. Irrigated-DC-pulses/maize-mai    0    255   255   
4       4     4216.0            4. Irrigated-sugarcane    0    100     0   
5       5   511076.0         5. Rainfed-DC-cotton-rice  255    255     0   
6       6    52925.0  6. Rainfed-DC-sorghum-chickpea/f  238    130   238   
7       7   125627.0         7. Rainfed-SC-rice-fallow  255    192   203   
8       8    78126.0  8. Rainfed-SC-pigeonpea/groundnu  160     32   240   
9       9    11579.0    9. Rainfed-SC-cotton/groundnut  255    255   224   
10     10    46416.0             10. Rainfed-SC-pulses  165     42    42   
11     11    28260.0             11. Rainfed-groundnut  210    180   140   
12     12   

In [11]:
import rasterio
import pandas as pd
import numpy as np

# Read the DBF CSV file
def read_dbf_csv(dbf_csv_file):
    dbf_data = pd.read_csv(dbf_csv_file)
    return dbf_data

# Read the raster file
def read_raster(raster_file):
    with rasterio.open(raster_file) as src:
        array = src.read(1)  # Read the first band
        transform = src.transform
        metadata = src.meta
    return array, transform, metadata

# Convert pixel coordinates to geographic coordinates
def pixel_to_coords(pixel_array, transform):
    rows, cols = pixel_array.shape
    x, y = np.indices((rows, cols))
    x_coords, y_coords = rasterio.transform.xy(transform, x, y)
    # Convert lists to numpy arrays
    x_coords = np.array(x_coords)
    y_coords = np.array(y_coords)
    return x_coords, y_coords

# Map raster values to crop types
def map_raster_to_crop_types(raster_array, dbf_data):
    value_to_crop = dict(zip(dbf_data['Value'], dbf_data['Class_Name']))
    vectorized_map = np.vectorize(value_to_crop.get)
    crop_type_array = vectorized_map(raster_array)
    return crop_type_array

# Convert to CSV with blank value removal
def data_to_csv(x_coords, y_coords, crop_type_array, output_csv_file):
    df = pd.DataFrame({
        'Longitude': x_coords.flatten(),
        'Latitude': y_coords.flatten(),
        'Crop_Type': crop_type_array.flatten()
    })
    
    # Remove rows where 'Crop_Type' is NaN or empty string
    df = df[df['Crop_Type'].notna() & (df['Crop_Type'] != '')]
    
    df.to_csv(output_csv_file, index=False)

# File paths

raster_file = 'C:\Anna_Data_D_files\sem6\z_capstone\Agriculture\A_Phase_1\datasets\DiCRA\Telengana\SOCIO-ECONOMIC\CROP_TYPE\RASTER\\01-01-2015.tif'
dbf_csv_file = 'C:\Anna_Data_D_files\sem6\z_capstone\Agriculture\A_Phase_2\code- review2\preprocessed_files\DICRA\Crop_Type_2015_dbf.csv'
output_csv_file = 'C:\Anna_Data_D_files\sem6\z_capstone\Agriculture\A_Phase_2\code- review2\preprocessed_files\DICRA\Crop_Type_output_2.csv'


# Process
dbf_data = read_dbf_csv(dbf_csv_file)
raster_array, transform, metadata = read_raster(raster_file)
x_coords, y_coords = pixel_to_coords(raster_array, transform)
crop_type_array = map_raster_to_crop_types(raster_array, dbf_data)
data_to_csv(x_coords, y_coords, crop_type_array, output_csv_file)


In [ ]:
# # File paths

# raster_file = 'C:\Anna_Data_D_files\sem6\z_capstone\Agriculture\A_Phase_1\datasets\DiCRA\Telengana\SOCIO-ECONOMIC\CROP_TYPE\RASTER\\01-01-2015.tif'
# dbf_csv_file = 'C:\Anna_Data_D_files\sem6\z_capstone\Agriculture\A_Phase_2\code- review2\preprocessed_files\DICRA\Crop_Type_2015_dbf.csv'
# output_csv_file = 'C:\Anna_Data_D_files\sem6\z_capstone\Agriculture\A_Phase_2\code- review2\preprocessed_files\DICRA\Crop_Type_output.csv'
